In [7]:
import numpy as np

<h4>Загрузка простого .csv файла с помощью Numpy</h4>

In [8]:
data = np.genfromtxt('data/les2_data_example.csv') # ('path/to/myfile.csv',delimiter=',',skiprows=0, dtype=float)
data

array([ 64.,  40., 107., ...,  59.,  65.,  62.])

<h4>Пример</h4>
Пусть N=30463.<br> 
Для заданной выборки рассчитаем среднее, медиану, дисперсию, стандартное отклонение, 1й и 3ю квартиль. <br>
Также рассчитаем 98.0%-ый доверительный интервал для <b>суммы</b> 

In [9]:
N = 30463
n = data.size # или len(data)
n

1015

Сначала рассчитаем оценку среднего значения выборки

In [10]:
y_summ = data.sum()
y_summ_2 = np.power(data,2).sum()
y_mean = y_summ/n
print(y_mean)
print(data.mean())

54.570443349753695
54.570443349753695


Далее рассчитаем оценку общего (суммарного значения) для генеральной совокупности

In [11]:
Y_summ = y_mean*N
Y_summ

1662379.4157635467

Рассчитаем долю отбора

In [12]:
f = n/N
f

0.033319108426615895

In [22]:
t_sum = 0
for i in range(len(data)):
    t_sum += (data[i]-y_mean)**2
print(t_sum)
t_sum/(len(data)-1)

488408.71330049203


481.66539773224065

Рассчитаем выборочную дисперсию

In [13]:
var = np.sum(np.power(data-y_mean,2)) / (n-1)
var

481.6653977322412

In [9]:
data.var(ddof=1)

481.6653977322412

In [11]:
#Будем использовать эту формулу
var = (y_summ_2 - y_summ**2/n) / (n-1)
var

481.6653977322412

И стандартное отклонение

In [12]:
std = np.sqrt(var)
std

21.946876719302026

In [13]:
data.std(ddof=1)

21.946876719302026

Стандартная ошибка среднего (насколько оценка среднее значение отклоняется от среднего)

In [14]:
std_mean_error = (std * np.sqrt(1-f)) / np.sqrt(n)
std_mean_error

0.6773003031231355

Стандартная ошибка суммарного значения (насколько оценка суммарного значения отклоняется от суммы ГС)

In [15]:
std_summ_error = N * (std * np.sqrt(1-f)) / np.sqrt(n)
std_summ_error

20632.599134040076

Для того, чтобы рассчитать доверительные интервалы нужно найти квантиль t уровня &alpha; нормального распределения N(0,1). В приведенной формуле, &alpha; - вероятность ошибки. Если нам необходимы 80% доверительные границы, то &alpha; = 20%.

In [16]:
from scipy.stats import norm

In [17]:
alpha =1-0.98

In [17]:
#Левая квантиль
t_2a=norm.ppf(alpha/2)
t_2a

-2.3263478740408408

In [18]:
#Правая квантиль
t_1_2a=norm.ppf(1-alpha/2)
t_1_2a

2.3263478740408408

Рассчитаем левую и правую границу доверительного интервала для <b>среднего значения</b>

In [19]:
Y_mean_left =y_mean + t_2a * std_mean_error
Y_mean_right =y_mean + t_1_2a * std_mean_error
(Y_mean_left,Y_mean_right)

(52.99480722949597, 56.14607947001142)

Видим, что t_2a и t_1_2a симметричны, поэтому можно так:

In [18]:
t = norm.ppf(1-alpha/2)
Y_mean_left =y_mean - t * std_mean_error
Y_mean_right =y_mean + t * std_mean_error
(Y_mean_left,Y_mean_right)

(52.99480722949597, 56.14607947001142)

Рассчитаем левую и правую границу доверительного интервала для <b>суммы</b>

In [19]:
Y_mean_left * N, Y_mean_right * N

(1614380.8126321356, 1710378.0188949578)

<h4>Найдем квартили</h4>

In [20]:
data.sort()
data

array([  9.,  13.,  15., ..., 184., 202., 215.])

In [21]:
len(data)

1015

Количество элементов <b>нечетное</b>. Вычисляем индекс по формуле для n = 2k + 1

In [22]:
k=int((n-1)/2) # n-1 т.к. нумерация индексов начинается с 0
k

507

In [23]:
q2 = data[k]
q2

51.0

In [24]:
q1_data = data[:k] # [0:k)
q3_data = data[k+1:] # (k:n]

элемент под номером 508 (507й индекс) делит выборку на 2 ровных части по 507 элементов. Он не включается ни в одну из этих частей

In [26]:
len(q1_data), len(q3_data)

(507, 507)

Проделаем то же для оставшихся наборов

In [27]:
k1 = int((len(q1_data) - 1) / 2)
k3 = int((len(q3_data) - 1) / 2)

In [28]:
q1 = q1_data[k1]
q3 = q3_data[k3]

In [30]:
print(q1, q2, q3) # по формуле
print(np.median(q1_data), np.median(data), np.median(q3_data)) # сравним с методом median
print(np.percentile(data, 25), np.percentile(data, 50), np.percentile(data, 75)) # сравним с методом percentile

39.0 51.0 64.0
39.0 51.0 64.0
39.0 51.0 64.0


<h4>Сохраняем в .csv</h4>
Используем библиотеку <b>pandas</b>

In [31]:
import pandas as pd

Создаем словарь со всеми необходимыми значениями. round() для округления до 3х знаков после запятой (в лабе не выполняем)

In [32]:
d = {'mean': round(y_mean, 3),
     'median': round(q2, 3),
     'var': round(var, 3),
     'std_value': round(std, 3),
     'quantile1': round(q1, 3),
     'quantile3': round(q3, 3),
     'CI_left': round(Y_mean_left * N, 3),
     'CI_right': round(Y_mean_right * N, 3)
    }
d

{'mean': 54.57,
 'median': 51.0,
 'var': 481.665,
 'std_value': 21.947,
 'quantile1': 39.0,
 'quantile3': 64.0,
 'CI_left': 1614380.813,
 'CI_right': 1710378.019}

Создаем <b>DataFrame</b>

In [41]:
df = pd.DataFrame(index=[0],data=d)
df

,mean,median,var,std_value,quantile1,quantile3,CI_left,CI_right
0,54.57,51.0,481.665,21.947,39.0,64.0,1614380.813,1710378.019


Сохраняем в файл .csv

In [42]:
df.to_csv('data/result.csv', index=False)

Как прочитать .csv файл с помощью pandas

In [45]:
df1 = pd.read_csv('data/les2_data_example.csv')
df1

,col_name
0,64
1,40
2,107
3,74
4,34
...,...
1010,77
1011,51
1012,59
1013,65


In [49]:
df1['col_name'].values

481.1908505423573

In [50]:
N = 676
n = 50
sample = []
for i in [(42, 23), (41, 4), (36, 1), (32, 1), (29, 1), (27, 2), (23, 1), (19, 1), (16, 2), (15, 2), (14, 1), (11, 1), (10, 1), (9, 1), (7, 1), (6, 3), (5, 2), (4, 1), (3, 1)]:
    sample.append(i)

sample

[(42, 23),
 (41, 4),
 (36, 1),
 (32, 1),
 (29, 1),
 (27, 2),
 (23, 1),
 (19, 1),
 (16, 2),
 (15, 2),
 (14, 1),
 (11, 1),
 (10, 1),
 (9, 1),
 (7, 1),
 (6, 3),
 (5, 2),
 (4, 1),
 (3, 1)]

In [51]:
samp = np.array(sample)
samp

array([[42, 23],
       [41,  4],
       [36,  1],
       [32,  1],
       [29,  1],
       [27,  2],
       [23,  1],
       [19,  1],
       [16,  2],
       [15,  2],
       [14,  1],
       [11,  1],
       [10,  1],
       [ 9,  1],
       [ 7,  1],
       [ 6,  3],
       [ 5,  2],
       [ 4,  1],
       [ 3,  1]])

In [58]:
y_summ = (samp[:, 0] * samp[:, 1]).sum()
y_summ

1471

In [60]:
mean = y_summ / n
mean

29.42

In [61]:
y_summ_2 = ((samp[:, 0] ** 2) * samp[:, 1]).sum()
y_summ_2

54497

In [62]:
var = (y_summ_2 - (y_summ**2) / n) / (n-1)
var

228.98326530612246

In [64]:
std = np.sqrt(var)
std

15.132193010470177

In [66]:
f = n/N
f

0.07396449704142012

In [68]:
y_mean_std = std * np.sqrt(1-f) / np.sqrt(n)
y_mean_std

2.0593524868601443

In [71]:
alpha = 1 - 0.8
t = norm.ppf(1-alpha/2)
t

1.2815515655446004

In [72]:
ci_left = N * mean - t*y_mean_std*N
ci_right = N * mean + t*y_mean_std*N
ci_left, ci_right

(18103.843511204403, 21671.9964887956)

In [6]:
import scipy as sp
import pandas as pd
import numpy as np

In [2]:
fams = sp.random.poisson(1.5, 50) + 1
fams

array([1, 2, 1, 2, 3, 1, 1, 1, 2, 2, 4, 8, 3, 1, 2, 3, 6, 1, 2, 4, 7, 3,
       3, 1, 3, 1, 2, 2, 2, 2, 4, 3, 2, 2, 4, 2, 2, 3, 2, 3, 3, 2, 2, 3,
       3, 1, 5, 3, 3, 2])

In [3]:
def get_earnings(f_size):
    r = min(2, f_size)
    rest = f_size - r
    return r * 800 + np.random.normal(0, 100, 1)[0] + rest * 250

In [7]:
df = pd.DataFrame([(f, get_earnings(f)) for f in fams], columns=['family_size', 'total_income'])
df

,family_size,total_income
0,1,809.117916
1,2,1620.075611
2,1,668.834529
3,2,1500.974265
4,3,1808.812849
5,1,799.643375
6,1,840.517909
7,1,738.816924
8,2,1771.015077
9,2,1631.995892


In [8]:
R = df['total_income'].sum() / df['family_size'].sum()
R

629.5902684264455

In [9]:
N=1000
n=len(df)
f=n/N
f

0.05

In [11]:
x = df['family_size'].values
y = df['total_income'].values

In [12]:
x

array([1, 2, 1, 2, 3, 1, 1, 1, 2, 2, 4, 8, 3, 1, 2, 3, 6, 1, 2, 4, 7, 3,
       3, 1, 3, 1, 2, 2, 2, 2, 4, 3, 2, 2, 4, 2, 2, 3, 2, 3, 3, 2, 2, 3,
       3, 1, 5, 3, 3, 2])

In [13]:
y

array([ 809.11791568, 1620.07561104,  668.83452884, 1500.97426487,
       1808.81284867,  799.6433753 ,  840.51790894,  738.81692423,
       1771.01507676, 1631.9958924 , 2187.89543415, 3041.88956852,
       1697.17068993,  839.95411792, 1608.27393465, 1812.433421  ,
       2594.9257043 ,  858.00484657, 1558.28904562, 2078.44598996,
       2945.1323621 , 1726.88403601, 1788.19981452,  871.22422293,
       2001.99370401,  891.39021715, 1738.73653893, 1590.5366521 ,
       1761.67415678, 1646.94129956, 2108.50054089, 1769.71167076,
       1615.21037409, 1489.61315361, 2107.16545587, 1501.52601135,
       1464.2417016 , 1666.34127775, 1683.05787954, 1961.08923307,
       1619.2349044 , 1557.85972549, 1643.29453513, 1790.12080115,
       1850.58362464,  898.79448536, 2279.77046144, 1970.11003758,
       1890.1641174 , 1550.54480088])

In [14]:
A = (x*x).sum() * R**2 + (y*y).sum() - 2*R*(x*y).sum()

In [15]:
B = A/(n-1)
B

242997.0079205231

In [17]:
S = np.sqrt(B)
S

492.9472668760048

In [19]:
(np.sqrt(1-f)*S)/(np.sqrt(n)*x.mean())

26.133881380109532